# Sentiment analysis

## I. Introduction 

Dans cette première partie, nous abordons la prédiction du sentiment des reviews Steam comme un problème de classification binaire. À partir du texte brut d’un avis, l’objectif est de prédire si l’utilisateur recommande le jeu (Recommended) ou non (Not recommended).

Nous mettons en place une baseline basée sur des méthodes classiques de traitement automatique du langage : le texte est vectorisé via TF-IDF (unigrams et bigrams), puis un classifieur linéaire (régression logistique) est entraîné sur l’ensemble d’entraînement. Les performances sont évaluées sur un ensemble de test à l’aide de métriques adaptées (accuracy, F1-score, et matrice de confusion).

Cette tâche est généralement considérée comme relativement accessible avec des méthodes classiques, car le vocabulaire employé dans les avis est fortement polarisé : de nombreux termes et expressions sont clairement associés à des recommandations positives ou négatives.

### I.1 Definition du problème

Les textes des reviews ne pouvant pas être traités directement par des algorithmes de machine learning, ils sont transformés en représentations numériques à l’aide de la méthode TF-IDF. Cette approche attribue à chaque mot un poids proportionnel à sa fréquence dans un document, tout en pénalisant les termes trop fréquents dans l’ensemble du corpus. Les mots les plus informatifs — fréquents dans un avis mais rares globalement — reçoivent ainsi un poids plus élevé, permettant au modèle de capturer efficacement le signal de sentiment présent dans le texte.

### I.2 Vectorisation avec TF-IDF

On commence par créer un dictionnaire à partir de tous les mots présents dans les reviews d'entrainement.

Ensuite chaque mot reçoit un indice. Cependant avant de passer à la vectorisation, on ne souhaite pas prendre en compte les mots très fréquents qui n'apportent pas/peu d'informations (comme "est", "le"...).

D'où l'introduction du principe de TF-IDF (term frequency * inverse document frequency) qui repose sur l'idée suivante : Un mot est important s'il est fréquent dans un document mais rare dans le corpus.

Les formules sont les suivantes : 

TF(t,d)= (nombre d’occurrences de t dans d​) / (nombre total de mots dans d)

IDF(t) = log(N / 1 + DF(t)), avec N = nb total de documents et DF(t) = nb de documents contenant le mot.

--> Exemples : "The" a un IDF faible alors que "unplayable" a un IDF élevé.

Ainsi avec ce principe, les mots fréquents dans le document mais rares globalement reçoivent un poids élevé.

Maintenant nous pouvons passer à la vectorisation : Chaque review est alors représentée par un vecteur réel de dimension égale à la taille du vocabulaire, dont les composantes correspondent aux scores TF-IDF des termes.
On obtient que les vecteurs sont souvent très grands et très sparses (creux).

Bien que cette représentation soit de grande dimension, elle reste exploitable en pratique grâce :
- à l’utilisation de matrices creuses (on ne stocke que les valeurs non nulles) ;
- au fait que les modèles classiques de NLP (régression logistique, Naive Bayes) sont optimisés pour ce type de représentation ;
- à des mécanismes de filtrage du vocabulaire (suppression des mots trop fréquents).


### I.3 Importance of Bigrams

Enfin l’utilisation de bigrams permet de capturer des expressions locales essentielles à l’analyse du sentiment, en particulier les phénomènes de négation et les expressions figées. Alors qu’un modèle basé uniquement sur des unigrams traite les mots de manière indépendante, l’ajout de bigrams permet de représenter explicitement des séquences telles que “not good” ou “worth it. Cette information contextuelle améliore significativement la capacité du modèle à distinguer les avis positifs et négatifs.

## II. Baseline model

Après avoir présenté le principe de vectorisation TF-IDF et l’intérêt des bigrams, nous implémentons une baseline de classification binaire. Le texte des reviews est transformé en représentations TF-IDF, puis un classifieur linéaire est entraîné pour prédire si une review recommande ou non le jeu. Les performances sont ensuite évaluées sur un ensemble de test.

## II.1 Setup des Variables

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/processed/reviews.csv")

TEXT_COL = "review_text"
TARGET_COL = "recommended"

# Features (texte) + target
X = df[TEXT_COL].astype(str)

# True/False -> 1/0
y = df[TARGET_COL].astype(int)

print("Label distribution:")
print(y.value_counts())
print(y.value_counts(normalize=True))

Label distribution:
recommended
1    3777
0     676
Name: count, dtype: int64
recommended
1    0.848192
0    0.151808
Name: proportion, dtype: float64


## II.2 Split train/test (stratifié)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


## II.3 Baseline TF-IDF

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

baseline_lr = Pipeline([
    ("tfidf", TfidfVectorizer(
        lowercase=True,
        stop_words="english",   # langue du texte
        ngram_range=(1, 2),     # unigrams + bigrams
        min_df=5,
        max_df=0.9,
        max_features=50_000
    )),
    ("clf", LogisticRegression(
        max_iter=2000
    ))
])

baseline_lr.fit(X_train, y_train)


,steps,"[('tfidf', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


## II.4 Evaluation 

In [7]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = baseline_lr.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\n=== Classification report ===")
print(classification_report(y_test, y_pred, digits=3))
print("\n=== Confusion matrix ===")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.8630751964085297

=== Classification report ===
              precision    recall  f1-score   support

           0      0.842     0.119     0.208       135
           1      0.864     0.996     0.925       756

    accuracy                          0.863       891
   macro avg      0.853     0.557     0.566       891
weighted avg      0.860     0.863     0.816       891


=== Confusion matrix ===
[[ 16 119]
 [  3 753]]


## II.5 Mots les plus discriminants 

In [8]:
import numpy as np

vectorizer = baseline_lr.named_steps["tfidf"]
clf = baseline_lr.named_steps["clf"]

feature_names = np.array(vectorizer.get_feature_names_out())
coefs = clf.coef_[0]

top_pos_idx = np.argsort(coefs)[-20:][::-1]  # pousse vers Recommended (1)
top_neg_idx = np.argsort(coefs)[:20]         # pousse vers Not recommended (0)

print("Top positive features (Recommended):")
for term, w in zip(feature_names[top_pos_idx], coefs[top_pos_idx]):
    print(f"{term:25s} {w:.3f}")

print("\nTop negative features (Not recommended):")
for term, w in zip(feature_names[top_neg_idx], coefs[top_neg_idx]):
    print(f"{term:25s} {w:.3f}")


Top positive features (Recommended):
best                      2.643
good                      2.395
love                      2.200
amazing                   1.670
10                        1.631
great                     1.438
cyberpunk                 1.367
awesome                   1.163
city                      1.150
fun                       1.122
peak                      1.115
10 10                     1.036
favorite                  0.995
best game                 0.991
masterpiece               0.978
funny                     0.958
greatest                  0.958
shooter                   0.932
truly                     0.919
beautiful                 0.912

Top negative features (Not recommended):
shit                      -2.952
cheaters                  -2.742
worst                     -2.707
boring                    -2.364
trash                     -2.191
toxic                     -2.098
match                     -1.928
waste                     -1.893
fix              

## III. Results

Bien que l’accuracy atteigne 86 %, l’analyse détaillée révèle un fort déséquilibre des performances entre les classes. Le modèle identifie presque parfaitement les reviews positives, mais échoue à détecter la majorité des avis négatifs, avec un rappel de seulement 12 % pour la classe Not recommended.

Cette situation s’explique par le déséquilibre du jeu de données, dominé par les avis positifs, qui conduit le modèle à privilégier la prédiction de la classe majoritaire. Ces résultats montrent que l’accuracy seule est insuffisante pour évaluer la qualité du modèle et soulignent l’importance de métriques telles que le rappel et le F1-score macro.